# Initializations

In [1]:
from astropy.table import Table, vstack
from astropy.io import ascii
import numpy as np

working_dir = '/Users/kmohamad/Documents/GitHub/SIP2019/'

# The Catalogs

In [2]:
main = {
    'VDGC':Table.read(working_dir + 'catalogs/VDGC_pPXF_2017_v2.fits'),
    'VUGC':Table.read(working_dir + 'catalogs/VUGC_pPXF_2017_v2.fits')
}

main_ref = {
    'VDGC_K':Table.read(working_dir + 'catalogs/VDGC_kinematic_prob.fits'),
    'VDGC_O':Table.read(working_dir + 'catalogs/orphanGCs.fits'),
    'VUGC_K':Table.read(working_dir + 'catalogs/SelectGCsTrue_kinematic_prob.fits'),
    'VUGC_O':Table.read(working_dir + 'catalogs/SelectGCsTrue2.fits'),
    'NUC':ascii.read(working_dir + 'catalogs/nuc.dat')
}

In [3]:
print(
    'VDGC:', str(len(main['VDGC'].colnames)), 
    'VUGC:', str(len(main['VUGC'].colnames))
)

VDGC: 27 VUGC: 24


In [4]:
set(main['VDGC'].colnames).difference(set(main['VUGC'].colnames))

{'KECKID',
 'ZERR',
 'ZERR_ne',
 'ZERR_pe',
 'ZHEL',
 'ZOBS',
 'ZOBS_ne',
 'ZOBS_pe',
 'ZQUAL',
 'ZSPECNUM'}

In [5]:
set(main['VUGC'].colnames).difference(set(main['VDGC'].colnames))

{'VERR', 'VERR_ne', 'VERR_pe', 'VHEL', 'VOBS', 'VOBS_ne', 'VOBS_pe'}

In [6]:
columns_to_change = [
    'ZERR', 'ZERR_ne', 'ZERR_pe', 
    'ZHEL', 'HELCOR',
    'ZOBS', 'ZOBS_ne', 'ZOBS_pe',
    'ABANDCOR', 'ABANDCOR_ne', 'ABANDCOR_pe'
]

for col in columns_to_change:
    main['VDGC'][col] = main['VDGC'][col] * 3e+5
    if col[0] is 'Z':
        main['VDGC'].rename_column(col, 'V' + col[1:])   
    
main['VUGC']['KECKID'] = '---'
main['VUGC']['ZQUAL'] = -101010
main['VUGC']['ZSPECNUM'] = -101010

main['VDGC']['CATALOG'] = 'VDGC'
main['VUGC']['CATALOG'] = 'VUGC'

In [7]:
for cat in ['VDGC', 'VUGC']:
    main[cat]['NUCLEUS'] = '--------'
    main[cat]['OBJ_TYPE'] = '--------'
    main[cat]['OBJ_PROB'] = -1

In [8]:
for cat in main:
    main[cat] = main[cat][sorted(main[cat].colnames)]

# Prepping Sub-Catalogs


In [9]:
for cat in main:
    main[cat] = main[cat][main[cat]['ZCONF'] == 1]

print([set(main[cat]['ZCONF']) for cat in main])

[{1.0}, {1.0}]


### DE and UDG GC Satellites

In [10]:
DE_GC = main['VDGC'][main['VDGC']['GCSAT'] == 'Y']
UDG_GC = main['VUGC'][main['VUGC']['GCSAT'] == 'Y']

non_udg = (UDG_GC['HOST'] != 'VLSB-B') & (UDG_GC['HOST'] != 'VLSB-D') & (UDG_GC['HOST'] != 'VCC0615')

DE_GC = vstack([DE_GC, UDG_GC[non_udg]])
UDG_GC.remove_rows(np.nonzero(non_udg))

DE_GC['OBJ_TYPE'] = 'DE_GC'
DE_GC['OBJ_PROB'] = 1
UDG_GC['OBJ_TYPE'] = 'UDG_GC'
UDG_GC['OBJ_PROB'] = 1

### IGC, M87 GC, and Milky Way Stars

In [11]:
def getGCType(catalog):
    df = catalog.to_pandas()
    obj_type = df.idxmax(axis = 1)
    probability = df.apply(max, axis = 1)
    
    obj_type[obj_type == 'p_star_V'] = 'Star'
    obj_type[obj_type == 'p_ICL_V'] = 'IGC'
    obj_type[obj_type == 'p_halo_V'] = 'M87_GC'
    
    return list(obj_type), list(probability)

In [12]:
non_sat = {
    'VDGC':main['VDGC'][main['VDGC']['GCSAT'] != 'Y'],
    'VUGC':main['VUGC'][main['VUGC']['GCSAT'] != 'Y']    
}

obj = {}
prob = {}

obj['VDGC'], prob['VDGC'] = getGCType(main_ref['VDGC_K'])
obj['VUGC'], prob['VUGC'] = getGCType(main_ref['VUGC_K'])

# One row is missing in the VUGC_O catalog. Add a row with UNK and -1 values for the particular SPEC1DNAME
print(set(non_sat['VUGC']['SPEC1DNAME']).difference(set(main_ref['VUGC_O']['SPEC1DNAME'])))

for cat in ['VDGC', 'VUGC']:
    non_sat[cat].sort('SPEC1DNAME')
    
    temp = Table()
    temp['SPEC1DNAME'] = main_ref[cat + '_O']['SPEC1DNAME']
    temp['OBJ_TYPE'] = obj[cat]
    temp['OBJ_PROB'] = prob[cat]
    
    if cat is 'VUGC':
        temp.add_row(['spec1d.vugc1.044.GCXO0504.fits.gz', 'UNK', -1])
    
    temp.sort('SPEC1DNAME')

    non_sat[cat]['OBJ_TYPE'] = temp['OBJ_TYPE']
    non_sat[cat]['OBJ_PROB'] = temp['OBJ_PROB']

{'spec1d.vugc1.044.GCXO0504.fits.gz'}


### Identifying Nuclei

In [13]:
ind = np.where(non_sat['VDGC']['VCC'] != '---')

DE_NUC = non_sat['VDGC'][ind].copy()
DE_NUC.sort('SPEC1DNAME')
DE_NUC['OBJ_TYPE'] = 'DE_NUC'
DE_NUC['OBJ_PROB'] = 1
DE_NUC['NUCLEUS'] = main_ref['NUC']['Nucleus']

non_sat['VDGC'].remove_rows(ind)

In [14]:
ind = np.where(non_sat['VUGC']['TARGTYPE'] == 'NUC')
non_sat['VUGC']['NUCLEUS'][ind] = 1

### A Unified Catalog

In [15]:
catalog = vstack([
    DE_GC, 
    UDG_GC, 
    non_sat['VDGC'], 
    non_sat['VUGC'],
    DE_NUC
])

In [16]:
# catalog.write('sip_catalog.csv', format = 'csv', overwrite = True)

In [17]:
catalog.colnames

['ABANDCOR',
 'ABANDCOR_ne',
 'ABANDCOR_pe',
 'CATALOG',
 'DEC',
 'GCSAT',
 'HELCOR',
 'HOST',
 'KECKID',
 'MASKNAME',
 'NUCLEUS',
 'OBJ_PROB',
 'OBJ_TYPE',
 'RA',
 'SLITNUM',
 'SN',
 'SPEC1DNAME',
 'TARGTYPE',
 'VCC',
 'VERR',
 'VERR_ne',
 'VERR_pe',
 'VHEL',
 'VOBS',
 'VOBS_ne',
 'VOBS_pe',
 'YHIGH',
 'YLOW',
 'ZCONF',
 'ZQUAL',
 'ZSPECNUM']

In [18]:
catalog.keep_columns(['SPEC1DNAME', 'ABANDCOR', 'ABANDCOR_ne', 'ABANDCOR_pe','VOBS', 'VOBS_ne', 'VOBS_pe'])

In [24]:
a = catalog.to_pandas()
col = a.columns.tolist()
col

['ABANDCOR',
 'ABANDCOR_ne',
 'ABANDCOR_pe',
 'SPEC1DNAME',
 'VOBS',
 'VOBS_ne',
 'VOBS_pe']

In [25]:
col.insert(0, col.pop(col.index('SPEC1DNAME')))

In [26]:
a = a.reindex(columns=col)

In [27]:
a['SPEC1DNAME'] = a['SPEC1DNAME'].str.decode('UTF-8')

In [32]:
a.to_csv('table_info_aband.csv', index = False)